In [1]:
import pandas as pd              
import numpy as np 
import tensorflow as tf

data = pd.read_csv("bible_data_set (with count and testament).csv")
data

,citation,book,chapter,verse,text,length (charcters),words,testament
0,Genesis 1:1,Genesis,1,1,In the beginning God created the heaven and th...,56,10,old
1,Genesis 1:2,Genesis,1,2,"And the earth was without form, and void; and ...",144,29,old
2,Genesis 1:3,Genesis,1,3,"And God said, Let there be light: and there wa...",56,11,old
3,Genesis 1:4,Genesis,1,4,"And God saw the light, that it was good: and G...",87,17,old
4,Genesis 1:5,Genesis,1,5,"And God called the light Day, and the darkness...",117,22,old
...,...,...,...,...,...,...,...,...
31097,Revelation 22:17,Revelation,22,17,"And the Spirit and the bride say, Come. And le...",170,33,new
31098,Revelation 22:18,Revelation,22,18,For I testify unto every man that heareth the ...,189,37,new
31099,Revelation 22:19,Revelation,22,19,And if any man shall take away from the words ...,209,44,new
31100,Revelation 22:20,Revelation,22,20,"He which testifieth these things saith, Surely...",97,16,new


In [2]:
import nltk
from nltk.stem import PorterStemmer

ps = PorterStemmer() 

vocabulary_size = 0
word2location = {}

def prepare_vocabulary(data):
    index = 0
    for sentance in data['text']:
        #sentance = sentance.lower()
        words = nltk.word_tokenize(sentance)
        for word in words:
            stemed_word = ps.stem(word)
            if stemed_word not in word2location:
                word2location[stemed_word] = index
                index += 1
    return index

def convert2vec(sentance):
    #sentance = sentance.lower()
    res_vec = np.zeros(vocabulary_size)
    words = nltk.word_tokenize(sentance)
    for word in words:
        stemed_word = ps.stem(word)
        if stemed_word in word2location:
            res_vec[word2location[stemed_word]]+=1
    return res_vec

vocabulary_size = prepare_vocabulary(data)
print("the size of the vocabulary is: ", vocabulary_size)
word2location

the size of the vocabulary is:  9421


{'In': 0,
 'the': 1,
 'begin': 2,
 'god': 3,
 'creat': 4,
 'heaven': 5,
 'and': 6,
 'earth': 7,
 '.': 8,
 'wa': 9,
 'without': 10,
 'form': 11,
 ',': 12,
 'void': 13,
 ';': 14,
 'dark': 15,
 'upon': 16,
 'face': 17,
 'of': 18,
 'deep': 19,
 'spirit': 20,
 'move': 21,
 'water': 22,
 'said': 23,
 'let': 24,
 'there': 25,
 'be': 26,
 'light': 27,
 ':': 28,
 'saw': 29,
 'that': 30,
 'it': 31,
 'good': 32,
 'divid': 33,
 'from': 34,
 'call': 35,
 'day': 36,
 'he': 37,
 'night': 38,
 'even': 39,
 'morn': 40,
 'were': 41,
 'first': 42,
 'a': 43,
 'firmament': 44,
 'in': 45,
 'midst': 46,
 'made': 47,
 'which': 48,
 'under': 49,
 'abov': 50,
 'so': 51,
 'second': 52,
 'gather': 53,
 'togeth': 54,
 'unto': 55,
 'one': 56,
 'place': 57,
 'dri': 58,
 'land': 59,
 'appear': 60,
 'sea': 61,
 'bring': 62,
 'forth': 63,
 'grass': 64,
 'herb': 65,
 'yield': 66,
 'seed': 67,
 'fruit': 68,
 'tree': 69,
 'after': 70,
 'hi': 71,
 'kind': 72,
 'whose': 73,
 'is': 74,
 'itself': 75,
 'brought': 76,
 'third'

In [8]:
import random

rand = []
for r in range (6220):
            ra = random.randrange(0, 31100)
            rand.append(ra)
            
train_x = []
train_y = []
test_x = []
test_y = []
for i in range(len(data['text'])):
    if(i not in rand):
        train_x.append(i)
        train_y.append(i)
        
    elif(i in rand):
        test_x.append(i)
        test_y.append(i)

In [ ]:
tf.compat.v1.disable_v2_behavior()

books = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy', 'Joshua', 'Judges',
            'Ruth', '1 Samuel', '2 Samuel', '1 Kings', '2 Kings', '1 Chronicles', '2 Chronicles',
            'Ezra', 'Nehemiah', 'Esther', 'Job', 'Psalms', 'Proverbs', 'Ecclesiastes',
            'Song of Solomon', 'Isaiah', 'Jeremiah', 'Lamentations', 'Ezekiel', 'Daniel',
            'Hosea', 'Joel', 'Amos', 'Obadiah', 'Jonah', 'Micah', 'Nahum', 'Habakkuk',
            'Zephaniah', 'Haggai',    'Zechariah',    'Malachi', 'Matthew', 'Mark', 'Luke', 'John', 'Acts', 'Romans', '1 Corinthians',
            '2 Corinthians', 'Galatians', 'Ephesians', 'Philippians', 'Colossians',
            '1 Thessalonians', '2 Thessalonians', '1 Timothy', '2 Timothy', 'Titus', 'Philemon',
            'Hebrews', 'James', '1 Peter', '2 Peter', '1 John', '2 John', '3 John', 'Jude',
            'Revelation']

def encode(line):
    res_vec = np.zeros(66)
    idx = books.index(data.iloc[line]['book'])
    res_vec[idx] = 1
    return res_vec

def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

features = vocabulary_size
categories = 66 #number of books
epsilon = 1e-12
x = tf.compat.v1.placeholder(tf.float32,[None,features])
y_ = tf.compat.v1.placeholder(tf.float32,[None,categories])
W = tf.compat.v1.Variable(tf.zeros([features,categories]))
b = tf.compat.v1.Variable(tf.zeros([categories]))

y = tf.nn.softmax(tf.matmul(x,W)+b)
loss = -tf.reduce_mean(y_*tf.compat.v1.log(y))

update = tf.compat.v1.train.GradientDescentOptimizer(0.00001).minimize(loss) #0.001

data_x = np.array([convert2vec(data.iloc[i]['text']) for i in train_x])
data_y = np.array([encode(i) for i in train_y])
test_data_x = np.array([convert2vec(data.iloc[i]['text']) for i in test_x])
test_data_y = np.array([encode(i) for i in test_y])
# data_x = np.array([convert2vec(data.iloc[i]['text']) for i in range(len(data['text']))])
# data_y = np.array([encode(i) for i in range(len(data['text']))])

sess = tf.compat.v1.Session()

sess.run(tf.compat.v1.global_variables_initializer())

for i in range(10000):
    sess.run(update, feed_dict = {x:data_x, y_:data_y})
    if i % 1 == 0 :
        print('Iteration:' , i , ' W:' , sess.run(W) , ' b:' , sess.run(b), ' loss:', loss.eval(session=sess, feed_dict = {x:data_x, y_:data_y}))
    

test1 = "In the beginning God created the heaven and the earth. "
t1 = np.array([convert2vec(test1)])
print("prediction for 1,1: " ,sess.run(y,feed_dict={x:t1}))

Iteration: 0  W: [[ 3.43785070e-11  1.05572018e-11  1.65125344e-11 ... -2.51747304e-11
  -2.51747338e-11 -1.92194195e-11]
 [ 7.29625249e-09  1.02679625e-08  7.27839211e-09 ... -4.65606131e-09
  -4.51313742e-09  4.89326912e-10]
 [ 1.92194715e-11 -4.60183584e-12 -1.05571550e-11 ... -1.05571550e-11
  -1.05571550e-11  1.92194281e-11]
 ...
 [-9.02320617e-14 -9.02320617e-14 -9.02320617e-14 ... -9.02320617e-14
  -9.02320617e-14  5.86508446e-12]
 [-9.02320617e-14 -9.02320617e-14 -9.02320617e-14 ... -9.02320617e-14
  -9.02320617e-14  5.86508446e-12]
 [-9.02320617e-14 -9.02320617e-14 -9.02320617e-14 ... -9.02320617e-14
  -9.02320617e-14  5.86508446e-12]]  b: [ 5.2737130e-09  3.5407317e-09  1.8970767e-09  3.9992591e-09
  2.3079687e-09  9.0849145e-10  6.5837213e-10 -1.8488614e-09
  1.5695227e-09  1.0335529e-09  1.5873898e-09  1.3730023e-09
  2.1769586e-09  1.7720039e-09 -8.9606850e-10 -2.6479866e-10
 -1.4915377e-09  2.9630507e-09  9.6031219e-09  2.0459492e-09
 -1.1818555e-09 -1.7118512e-09  4.0647

Iteration: 5  W: [[ 2.06271028e-10  6.33431987e-11  9.90752064e-11 ... -1.51048368e-10
  -1.51048368e-10 -1.15316499e-10]
 [ 4.37775114e-08  6.16077713e-08  4.36703509e-08 ... -2.79363643e-08
  -2.70788156e-08  2.93596658e-09]
 [ 1.15316826e-10 -2.76110176e-11 -6.33429212e-11 ... -6.33429004e-11
  -6.33429004e-11  1.15316576e-10]
 ...
 [-5.41392452e-13 -5.41392397e-13 -5.41392289e-13 ... -5.41392018e-13
  -5.41392018e-13  3.51905033e-11]
 [-5.41392452e-13 -5.41392397e-13 -5.41392289e-13 ... -5.41392018e-13
  -5.41392018e-13  3.51905033e-11]
 [-5.41392343e-13 -5.41392343e-13 -5.41392289e-13 ... -5.41392235e-13
  -5.41392235e-13  3.51905033e-11]]  b: [ 3.16422764e-08  2.12443911e-08  1.13824594e-08  2.39955558e-08
  1.38478127e-08  5.45094858e-09  3.95023259e-09 -1.10931682e-08
  9.41713729e-09  6.20131724e-09  9.52433865e-09  8.23801383e-09
  1.30617526e-08  1.06320233e-08 -5.37641087e-09 -1.58879176e-09
 -8.94922536e-09  1.77783033e-08  5.76187276e-08  1.22756951e-08
 -7.09113301e-09 -

Iteration: 10  W: [[ 3.7816339e-10  1.1612908e-10  1.8163782e-10 ... -2.7692199e-10
  -2.7692199e-10 -2.1141358e-10]
 [ 8.0258758e-08  1.1294756e-07  8.0062293e-08 ... -5.1216659e-08
  -4.9644481e-08  5.3826059e-09]
 [ 2.1141404e-10 -5.0620220e-11 -1.1612869e-10 ... -1.1612863e-10
  -1.1612863e-10  2.1141373e-10]
 ...
 [-9.9255359e-13 -9.9255337e-13 -9.9255315e-13 ... -9.9255207e-13
  -9.9255218e-13  6.4515927e-11]
 [-9.9255359e-13 -9.9255337e-13 -9.9255315e-13 ... -9.9255207e-13
  -9.9255218e-13  6.4515927e-11]
 [-9.9255272e-13 -9.9255272e-13 -9.9255250e-13 ... -9.9255207e-13
  -9.9255207e-13  6.4515934e-11]]  b: [ 5.80108370e-08  3.89480483e-08  2.08678408e-08  4.39918466e-08
  2.53876564e-08  9.99340521e-09  7.24209270e-09 -2.03374739e-08
  1.72647532e-08  1.13690817e-08  1.74612875e-08  1.51030246e-08
  2.39465479e-08  1.94920453e-08 -9.85675275e-09 -2.91278490e-09
 -1.64069132e-08  3.25935545e-08  1.05634328e-07  2.25054411e-08
 -1.30004105e-08 -1.88303648e-08  4.47125039e-08  4.7

Iteration: 15  W: [[ 5.50055446e-10  1.68914785e-10  2.64200190e-10 ... -4.02795602e-10
  -4.02795602e-10 -3.07510656e-10]
 [ 1.16739976e-07  1.64287329e-07  1.16454231e-07 ... -7.44969526e-08
  -7.22101490e-08  7.82923859e-09]
 [ 3.07511072e-10 -7.36294498e-11 -1.68914452e-10 ... -1.68914382e-10
  -1.68914382e-10  3.07510878e-10]
 ...
 [-1.44371505e-12 -1.44371461e-12 -1.44371407e-12 ... -1.44371169e-12
  -1.44371180e-12  9.38413583e-11]
 [-1.44371505e-12 -1.44371461e-12 -1.44371407e-12 ... -1.44371169e-12
  -1.44371180e-12  9.38413583e-11]
 [-1.44371331e-12 -1.44371331e-12 -1.44371299e-12 ... -1.44371201e-12
  -1.44371201e-12  9.38413652e-11]]  b: [ 8.43794012e-08  5.66517002e-08  3.03532239e-08  6.39881392e-08
  3.69274957e-08  1.45358605e-08  1.05339533e-08 -2.95817770e-08
  2.51123691e-08  1.65368466e-08  2.53982364e-08  2.19680363e-08
  3.48313378e-08  2.83520674e-08 -1.43370933e-08 -4.23677848e-09
 -2.38646010e-08  4.74088147e-08  1.53649950e-07  3.27351870e-08
 -1.89096916e-08 

Iteration: 20  W: [[ 7.2194739e-10  2.2170037e-10  3.4676240e-10 ... -5.2866916e-10
  -5.2866916e-10 -4.0360779e-10]
 [ 1.5322121e-07  2.1562710e-07  1.5284617e-07 ... -9.7777246e-08
  -9.4775814e-08  1.0275868e-08]
 [ 4.0360804e-10 -9.6638711e-11 -2.2170024e-10 ... -2.2170007e-10
  -2.2170007e-10  4.0360804e-10]
 ...
 [-1.8948771e-12 -1.8948764e-12 -1.8948753e-12 ... -1.8948714e-12
  -1.8948714e-12  1.2316678e-10]
 [-1.8948771e-12 -1.8948764e-12 -1.8948753e-12 ... -1.8948714e-12
  -1.8948714e-12  1.2316678e-10]
 [-1.8948747e-12 -1.8948742e-12 -1.8948738e-12 ... -1.8948721e-12
  -1.8948721e-12  1.2316680e-10]]  b: [ 1.1074796e-07  7.4355370e-08  3.9838604e-08  8.3984446e-08
  4.8467331e-08  1.9078316e-08  1.3825813e-08 -3.8826080e-08
  3.2959981e-08  2.1704606e-08  3.3335180e-08  2.8833043e-08
  4.5716124e-08  3.7212089e-08 -1.8817438e-08 -5.5607727e-09
 -3.1322287e-08  6.2224075e-08  2.0166559e-07  4.2964942e-08
 -2.4818974e-08 -3.5948879e-08  8.5360234e-08  9.1488189e-08
 -3.2697226e

...

Iteration: 395  W: [[ 1.36136995e-08  4.18046975e-09  6.53884769e-09 ... -9.96912863e-09
  -9.96912952e-09 -7.61087282e-09]
 [ 2.88926663e-06  4.06606250e-06  2.88221258e-06 ... -1.84378177e-06
  -1.78717528e-06  1.93772010e-07]
 [ 7.61080976e-09 -1.82240456e-09 -4.18067980e-09 ... -4.18059454e-09
  -4.18059676e-09  7.61087282e-09]
 ...
 [-3.57331838e-11 -3.57330415e-11 -3.57326807e-11 ... -3.57312825e-11
  -3.57313033e-11  2.32257391e-09]
 [-3.57331838e-11 -3.57330415e-11 -3.57326807e-11 ... -3.57312825e-11
  -3.57313033e-11  2.32257391e-09]
 [-3.57325072e-11 -3.57323234e-11 -3.57322540e-11 ... -3.57316225e-11
  -3.57316225e-11  2.32257391e-09]]  b: [ 2.0883879e-06  1.4021331e-06  7.5124308e-07  1.5837105e-06
  9.1395219e-07  3.5976302e-07  2.6071561e-07 -7.3214460e-07
  6.2153038e-07  4.0928776e-07  6.2860721e-07  5.4370702e-07
  8.6207137e-07  7.0171131e-07 -3.5484399e-07 -1.0486015e-07
 -5.9064826e-07  1.1733697e-06  3.8028381e-06  8.1019721e-07
 -4.6801364e-07 -6.7789034e-07  1.60

Iteration: 400  W: [[ 1.3785589e-08  4.2332520e-09  6.6214083e-09 ... -1.0095001e-08
  -1.0095002e-08 -7.7069693e-09]
 [ 2.9257465e-06  4.1174012e-06  2.9186037e-06 ... -1.8670614e-06
  -1.8097404e-06  1.9621862e-07]
 [ 7.7069062e-09 -1.8454156e-09 -4.2334665e-09 ... -4.2333790e-09
  -4.2333812e-09  7.7069693e-09]
 ...
 [-3.6184385e-11 -3.6184226e-11 -3.6183865e-11 ... -3.6182432e-11
  -3.6182453e-11  2.3518993e-09]
 [-3.6184385e-11 -3.6184226e-11 -3.6183865e-11 ... -3.6182432e-11
  -3.6182453e-11  2.3518993e-09]
 [-3.6183691e-11 -3.6183490e-11 -3.6183421e-11 ... -3.6182772e-11
  -3.6182772e-11  2.3518993e-09]]  b: [ 2.1147564e-06  1.4198370e-06  7.6072854e-07  1.6037069e-06
  9.2549197e-07  3.6430552e-07  2.6400741e-07 -7.4138876e-07
  6.2937789e-07  4.1445554e-07  6.3654426e-07  5.5057200e-07
  8.7295604e-07  7.1057121e-07 -3.5932439e-07 -1.0618414e-07
 -5.9810583e-07  1.1881848e-06  3.8508529e-06  8.2042703e-07
 -4.7392280e-07 -6.8644954e-07  1.6299693e-06  1.7469931e-06
 -6.2435839

Iteration: 405  W: [[ 1.3957478e-08  4.2860342e-09  6.7039689e-09 ... -1.0220874e-08
  -1.0220875e-08 -7.8030657e-09]
 [ 2.9622263e-06  4.1687399e-06  2.9549949e-06 ... -1.8903411e-06
  -1.8323055e-06  1.9866523e-07]
 [ 7.8030027e-09 -1.8684267e-09 -4.2862531e-09 ... -4.2861634e-09
  -4.2861656e-09  7.8030657e-09]
 ...
 [-3.6635587e-11 -3.6635410e-11 -3.6635049e-11 ... -3.6633582e-11
  -3.6633602e-11  2.3812248e-09]
 [-3.6635587e-11 -3.6635410e-11 -3.6635049e-11 ... -3.6633582e-11
  -3.6633602e-11  2.3812248e-09]
 [-3.6634876e-11 -3.6634657e-11 -3.6634588e-11 ... -3.6633922e-11
  -3.6633922e-11  2.3812248e-09]]  b: [ 2.14112492e-06  1.43754085e-06  7.70214001e-07  1.62370327e-06
  9.37031757e-07  3.68848021e-07  2.67299214e-07 -7.50632921e-07
  6.37225412e-07  4.19623319e-07  6.44481304e-07  5.57436977e-07
  8.83840698e-07  7.19431114e-07 -3.63804787e-07 -1.07508129e-07
 -6.05563400e-07  1.20299990e-06  3.89886736e-06  8.30656859e-07
 -4.79831954e-07 -6.95008737e-07  1.65029314e-06  1.

Iteration: 410  W: [[ 1.41293670e-08  4.33881642e-09  6.78652956e-09 ... -1.03467466e-08
  -1.03467475e-08 -7.89916221e-09]
 [ 2.99870612e-06  4.22007861e-06  2.99138605e-06 ... -1.91362074e-06
  -1.85487067e-06  2.01111845e-07]
 [ 7.89909915e-09 -1.89143834e-09 -4.33903979e-09 ... -4.33894787e-09
  -4.33895009e-09  7.89916221e-09]
 ...
 [-3.70867885e-11 -3.70865942e-11 -3.70862334e-11 ... -3.70847311e-11
  -3.70847519e-11  2.41055020e-09]
 [-3.70867885e-11 -3.70865942e-11 -3.70862334e-11 ... -3.70847311e-11
  -3.70847519e-11  2.41055020e-09]
 [-3.70860599e-11 -3.70858240e-11 -3.70857546e-11 ... -3.70850711e-11
  -3.70850711e-11  2.41055020e-09]]  b: [ 2.16749345e-06  1.45524473e-06  7.79699462e-07  1.64369965e-06
  9.48571540e-07  3.73390520e-07  2.70591016e-07 -7.59877082e-07
  6.45072930e-07  4.24791097e-07  6.52418350e-07  5.64301956e-07
  8.94725360e-07  7.28291013e-07 -3.68285185e-07 -1.08832118e-07
 -6.13020973e-07  1.21781500e-06  3.94688186e-06  8.40886685e-07
 -4.85741111e-07

Iteration: 415  W: [[ 1.4301256e-08  4.3915986e-09  6.8690902e-09 ... -1.0472619e-08
  -1.0472620e-08 -7.9952587e-09]
 [ 3.0351860e-06  4.2714173e-06  3.0277772e-06 ... -1.9369004e-06
  -1.8774358e-06  2.0355846e-07]
 [ 7.9951956e-09 -1.9144499e-09 -4.3918265e-09 ... -4.3917323e-09
  -4.3917345e-09  7.9952587e-09]
 ...
 [-3.7537990e-11 -3.7537778e-11 -3.7537418e-11 ... -3.7535881e-11
  -3.7535901e-11  2.4398756e-09]
 [-3.7537990e-11 -3.7537778e-11 -3.7537418e-11 ... -3.7535881e-11
  -3.7535901e-11  2.4398756e-09]
 [-3.7537244e-11 -3.7536991e-11 -3.7536921e-11 ... -3.7536221e-11
  -3.7536221e-11  2.4398756e-09]]  b: [ 2.1938620e-06  1.4729486e-06  7.8918492e-07  1.6636960e-06
  9.6011138e-07  3.7793302e-07  2.7388282e-07 -7.6912124e-07
  6.5292045e-07  4.2995887e-07  6.6035540e-07  5.7116694e-07
  9.0561002e-07  7.3715091e-07 -3.7276558e-07 -1.1015611e-07
 -6.2047854e-07  1.2326301e-06  3.9948964e-06  8.5111651e-07
 -4.9165027e-07 -7.1212713e-07  1.6909407e-06  1.8123420e-06
 -6.4771336

we now encoded every book to one hot vector.
also we have the bag of words.

right now we see that our model is weak almost all numbers are equal, so we need to improve our model,
but after alot of epochs, the correct prediction get a little more points than other. 

In [9]:
data_x

array([[1., 3., 1., ..., 0., 0., 0.],
       [0., 6., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [29]:
data_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])